In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Apply the trained model to the unlabelled data as transfer learning. 
# As movie reviews and book reviews are similar in vocabulary, structure and sentiment, this is a valid transfer. 
# I could not use the imdb dataset, which is labelled for the purpose of this analysis as I am interested 
# in longitudinal effects which cannot be shown in the 25000 labelled reviews from that set.

In [4]:
# load the model
reloaded_model = tf.saved_model.load(r'D:\Programming\LHL-data-notes\Final\training\imdb_bert')

In [317]:
# load the csv it should be applied to
dfnum = 'D:\Programming\LHL-data-notes\Final\datachunked\Enronset200000.csv'
df = pd.read_csv(r'D:\Programming\LHL-data-notes\Final\datachunked\Enronset200000.csv')

In [318]:
df.head()

,Unnamed: 0,book_id,rating,review_text,original_publication_month,text_reviews_count,original_publication_year,original_title,rating_dist,original_publication_day,ratings_count,rev_year,rev_month,month_diff,time_cat
0,278821,8407173,4,Brilliant!! So well written I felt like I was ...,3,2127.0,2009,Jasper Jones,5:6929|4:7739|3:3175|2:822|1:298|total:18963,31.0,18963.0,2014,1,58,5.0
1,278822,13477676,4,what a great book..! some kids don't have the ...,1,4827.0,2013,"Forgive Me, Leonard Peacock",5:10353|4:12413|3:6968|2:1919|1:650|total:32303,1.0,32303.0,2014,5,16,2.0
2,278823,99561,2,** spoiler alert ** \n It really deserves more...,3,47959.0,2005,Looking for Alaska,5:368304|4:269778|3:151939|2:46001|1:17751|tot...,3.0,853773.0,2016,6,135,200.0
3,278824,11387515,4,My Precepts; \n Children will be raised with v...,2,46149.0,2012,Wonder,5:245331|4:114225|3:36912|2:8351|1:4755|total:...,14.0,409574.0,2014,1,23,2.0
4,278825,11870085,5,A Brilliant piece of work!,1,142351.0,2012,The Fault in Our Stars,5:1351908|4:723931|3:341290|2:96986|1:50541|to...,10.0,2564656.0,2013,9,20,2.0


In [33]:
# need a function to apply the model to each row in my dataframe and create a column called BERT_sentiment
def sentiment_str(string,reloaded_model):
    string = [string]
    results = tf.sigmoid(reloaded_model(tf.constant(string)))
    return (results[0][0]).numpy()

In [319]:
# apply it to the loaded df
df['BERT_sent'] = df.progress_apply(lambda x: sentiment_str(x['review_text'], reloaded_model), 
                        axis=1)

100%|██████████| 100001/100001 [14:49<00:00, 112.41it/s]


In [320]:
df.tail()

,Unnamed: 0,book_id,rating,review_text,original_publication_month,text_reviews_count,original_publication_year,original_title,rating_dist,original_publication_day,ratings_count,rev_year,rev_month,month_diff,time_cat,BERT_sent
99996,418555,8882815,4,Good starting point :3,8,260.0,2010,The Death of Joan of Arc,5:1755|4:1607|3:1371|2:341|1:63|total:5137,24.0,5137.0,2013,5,33,5.0,0.637997
99997,418556,22628,2,"""Honestly, not a book to read.""",2,47826.0,1999,The Perks of Being a Wallflower,5:470357|4:303549|3:146312|2:39469|1:14674|tot...,1.0,974361.0,2013,4,170,200.0,0.364675
99998,418557,13262783,5,"""Not just your typical love story. The Best.""",8,15834.0,2012,Every Day,5:50986|4:48736|3:26868|2:8171|1:3085|total:13...,28.0,137846.0,2013,6,10,1.0,0.953416
99999,418558,6567017,3,Not that so catchy ...,4,16071.0,2010,"Will Grayson, Will Grayson",5:82524|4:89498|3:62094|2:19663|1:8463|total:2...,6.0,262242.0,2013,3,35,5.0,0.336815
100000,418559,11870085,5,I rarely read or watch anything that I know wi...,1,142351.0,2012,The Fault in Our Stars,5:1351908|4:723931|3:341290|2:96986|1:50541|to...,10.0,2564656.0,2017,5,64,120.0,0.408756


In [321]:
csvnum = dfnum[-10:-9]



In [322]:
csvnum

'2'

In [316]:
# save it to a csv
df.to_csv(f'datachunksent/sentiment_{csvnum}.csv')